In [1]:
import glob
import os
from collections import Counter

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input, concatenate
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from mutagen.easyid3 import EasyID3

Using TensorFlow backend.


In [2]:
def makeArtistList():
    path = "C:/Users/Takumi/Desktop/statistic/data/mp3/composer/"
    l = ['Beethoven', 'Franz Joseph Haydn', 'J.S. Bach']
    artist_list = []
    for name in l:
        print('{} process start.'.format(name))
        p = path + name + '/cut/'
        file_count = len(os.listdir(p))
        l2 = [name] * file_count
        artist_list.extend(l2)

    counter = Counter(artist_list)
    print(counter)
    
    return artist_list

In [3]:
artist_list = makeArtistList()

Beethoven process start.
Franz Joseph Haydn process start.
J.S. Bach process start.
Counter({'J.S. Bach': 7277, 'Beethoven': 2597, 'Franz Joseph Haydn': 705})


In [5]:
def readwave(file):
    wav, samplerate = librosa.load(file)
    return wav, samplerate


def MelFilterBank(y, sr):
    return librosa.feature.melspectrogram(y=y, sr=sr)

In [7]:
path = "C:/Users/Takumi/Desktop/statistic/data/mp3/composer/"
name_list = ['Beethoven', 'Franz Joseph Haydn', 'J.S. Bach']
data_list = []
for name in name_list:
    print('{} process start.'.format(name))
    p = path + name + '/cut/'
    wav_list = sorted(list(glob.glob(p + "*.wav")))
    data_list.extend(wav_list)
print(len(data_list))

Beethoven process start.
Franz Joseph Haydn process start.
J.S. Bach process start.
10579


In [ ]:
data_x = np.empty([128, 1292])
for i in range(len(data_list)):
    wav, sr = readwave(data_list[i])
#     print(wav.shape)
#     print(sr)
    t = np.arange(0, len(wav)) / sr
    m = MelFilterBank(wav, sr)
#     print(m)
#     print(m.shape)
#     if not i:
#         data_x = np.stack([data_x, m], 0)
#     else:
#         data_x = np.r_[data_x, m]
    data_x = np.r_['0, 3, -1', data_x, m]
    
    if not i % 1000:
        print(data_x.shape)

(2, 128, 1292)


In [ ]:
data_x = np.delete(data_x, 0, 0)

In [ ]:
def makeLSTMmodel(hidden=128, input_shape=(128,)):
    model = Sequential()
    model.add(LSTM(hidden, input_shape=input_shape))
    model.add(Dense(128))
    model.compile(loss="categorical_crossentropy", optimezer="adam")
    return model

In [ ]:
model = makeLSTMmodel()
model.fit(data_x, artist_list, batch_size=64, epochs=50, callbacks=[EarlyStopping(), validation_split=0.2])